In [ ]:
#! pip install geopandas
#! pip uninstall astrovision
! pip install astrovision
! pip install folium

In [ ]:
from astrovision.plot.plot_utils import make_mosaic
from astrovision.data import SatelliteImage, SegmentationLabeledSatelliteImage
import folium

import os
import s3fs
from tqdm import tqdm
import re
import numpy as np

import geopandas as gpd
import pandas as pd

from pyproj import Transformer
from shapely.geometry import Polygon


import rasterio
from rasterio.features import rasterize, shapes

import matplotlib.pyplot as plt

## Test API

In [ ]:
%env MLFLOW_TRACKING_URI=https://projet-slums-detection-128833.user.lab.sspcloud.fr
%env MLFLOW_S3_ENDPOINT_URL=https://minio.lab.sspcloud.fr

In [ ]:
%env AWS_ACCESS_KEY_ID=projet-slums-key
%env AWS_SECRET_ACCESS_KEY=

del os.environ["AWS_SESSION_TOKEN"]

In [ ]:
import requests

image_path = "projet-slums-detection/golden-test/patchs/segmentation/PLEIADES/MAYOTTE_CLEAN/2022/250/ORT_976_2022_0524_8587_U38S_8Bits_0011.jp2"
# image_path = "projet-slums-detection/data-preprocessed/patchs/BDTOPO/segmentation/PLEIADES/GUADELOUPE/2022/250/train/ORT_971_2022_0627_1797_U20N_8bits_0037.jp2"
# image_path = "projet-slums-detection/data-preprocessed/patchs/BDTOPO/segmentation/PLEIADES/GUADELOUPE/2022/250/train/ORT_971_2022_0627_1799_U20N_8bits_0049.jp2"
# image_path = "projet-slums-detection/data-preprocessed/patchs/BDTOPO/segmentation/PLEIADES/GUADELOUPE/2022/250/train/ORT_971_2022_0627_1801_U20N_8bits_0042.jp2"

response = requests.get(
    "https://satellite-images-inference.lab.sspcloud.fr/predict", params={"image": image_path}
)

lsi = SegmentationLabeledSatelliteImage(
    SatelliteImage.from_raster(
        file_path=f"/vsis3/{image_path}",
        dep=None,
        date=None,
        n_bands=3,
    ),
    np.array(response.json()["mask"]),
)
plot = lsi.plot(bands_indices=[0, 1, 2])

# lsi.label

## Test make_mosaic

In [ ]:
si = SatelliteImage.from_raster(
    file_path=f"/vsis3/{image_path}",
    dep=None,
    date=None,
    n_bands=3,
)

liste_si = si.split(250)
si2 = make_mosaic(liste_si, [0, 1, 2])
si2.array = si2.array.astype(int)
si2.plot([0, 1, 2])

### Chargement des îlots

Ya plus les IOU ?
stocvkage moyenne variances dans les artifact MLFlow
Ici je vais charger une table de contour géométriique d'ilot et créer une fonction qui à partir d'un contour polygonal d'îlot  et d'un modèle :

1) récupère les images s'intersectant avec l'îlot (une année donnée) via leur nom (cooronnées du coin incluse dans le nom)
2) un modèle vient faire des prédictions 01 ou en proba sur les images récupérées
3) On somme les probabilités ou les 1 contenus dans l'ilôt (partie la plus subtile ? peut on cropper un raster ? je l'espère)
4) on peut donc attribuer à l'îlot uner surface batie prédite par le modèle
5) se permettre une représentation cartographique en représentant sur un même graphe le fond de carte donné par les patchs qui entour e complètement l'ilot (zone rectangulaire), les prédictions du modèle, et lîlot en question (ou bien d'une prédiction de réseau de neurones en probabilités) somme les1 du masque dans le contour de l'ilot

Je travaille à partir du jeu de test d'images de Mayotte dans un premeir temps ppour avoir des masques prédits hors modèle et des ilots martiniques

Petites remarques comme ça : Quand on est en prod on veut toutes les images du coup dans l'idée plutôt chopper les grosses images qui s'intersectent dans l'ilot, les spliter faire les prédictions et ne garder que les points dans l'ilot : moralité On repart des grosses images m^me si l'intersection est moins fine 
Problème d'EPSG

PB : l'installation de requirements marche pas

 - ok dans  un premier temps je vais prendre l'le nom de l'image  récupérer les coordonnées
 - les transformer en point
 - et trouver quel ilot intersecte l'image


mc cp --recursive  s3/projet-slums-detection/ilots/ ilots

In [ ]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://" + "minio.lab.sspcloud.fr"})
fs.download(rpath="projet-slums-detection/ilots", lpath="ilots", recursive=True)

In [ ]:
# Load a shapefile
fp = "ilots/ilots_976.shp"
ilots_mayotte = gpd.read_file(fp)
print(ilots_mayotte.head())

# chargement liste images
bucket_name = "projet-slums-detection"
directory_path = "data-raw/PLEIADES/MAYOTTE/2022/"

full_path = f"{bucket_name}/{directory_path}"
files = fs.ls(full_path)

print(files[:6])

Je vais sélectionner les images qui tombent dans un ilot (ie où il y a intersection) et voir si je ne peux pas mettre 0 partou à l'extérieur de l'ilot

In [ ]:
# Function to extract coordinates from filename, adjusted for meters
def extract_coordinates(filename):
    matches = re.findall(r"\d{4}", filename)
    if len(matches) >= 2:
        return int(matches[-2]) * 1000, int(matches[-1]) * 1000
    return None, None


# Initialize the transformer from EPSG:4471 to WGS84
transformer = Transformer.from_crs("EPSG:4471", "EPSG:4326")

# Define the length of the square's side (in meters)
side_length = 1000  # Adjust as needed

# Process filenames and transform coordinates to create squares
data = []
for filename in files:
    x, y = extract_coordinates(filename)

    if x and y:
        # Define the square's corners in EPSG:4471
        top_left = (x, y)
        top_right = (x + side_length, y)
        bottom_right = (x + side_length, y - side_length)
        bottom_left = (x, y - side_length)

        # Create a polygon from these corners
        square = Polygon([top_left, top_right, bottom_right, bottom_left])

        # Transform each corner to latitude and longitude
        transformed_corners = [
            (lat, lon)
            for lon, lat in [transformer.transform(*corner) for corner in square.exterior.coords]
        ]

        # Create a new polygon with the transformed coordinates
        square_lat_lon = Polygon(transformed_corners)

        # Store the filename and the polygon
        data.append({"filename": filename, "geometry": square_lat_lon})

# Create a DataFrame
df = pd.DataFrame(data)

filename_to_polygon = gpd.GeoDataFrame(df, geometry="geometry", crs=4326)
filename_to_polygon.crs = "EPSG:4326"
# This DataFrame now contains the filenames and the corresponding square polygons in latitude and longitude
filename_to_polygon.head()

In [ ]:
intersection_ilot_filename = gpd.sjoin(
    ilots_mayotte, filename_to_polygon, how="inner", predicate="intersects"
)

intersection_ilot_filename.groupby(["depcom", "code_ilot"]).size().reset_index(
    name="count"
).sort_values(by="count", ascending=False).to_csv("comptage_ilots.csv", index=False)

intersection_ilot_filename

## Traitement d'un îlot :

- A un ilot correspond une liste de grosses images.
- On split chacune de ces grosses images en 64 images, on applique le modèle dessus et on refusionne l'ensemble (images et masques)
- ensuite je détermine le masque ilot qui donne l'apaprtenance de chaque pixel a l'ilot sur la grosse image ce qui me permet d'extirper le masque ilot


Je commence avec l'ilot 824 et ses 3 images

In [ ]:
def construire_raster_ilot(polygon, filename):
    # je recupere le carré de la grosse image sous forme de polygone
    polygon_image = filename_to_polygon.loc[
        filename_to_polygon["filename"] == filename
    ].geometry.iloc[0]

    # intersection avec l'ilot en cours
    intersection_ilot_image_polygon = polygon_image.intersection(polygon)

    # pour visualiser seulement :
    # multipolygone_union = MultiPolygon([polygon_image, intersection_ilot_image_polygon])
    # multipolygone_union

    # rasterio.transform.from_bounds
    x_min, y_min, x_max, y_max = polygon_image.bounds
    transform = rasterio.transform.from_bounds(x_min, y_min, x_max, y_max, 2000, 2000)

    valeur_polygone = 1

    # Rasteriser le polygone actuel avec une valeur unique
    raster_polygone = rasterize(
        [(intersection_ilot_image_polygon, valeur_polygone)],
        out_shape=(2000, 2000),
        transform=transform,
    )

    return raster_polygone


def representer_bout_ilot(polygon, filename):
    raster_image = construire_raster_ilot(polygon, filename)

    si = SatelliteImage.from_raster(
        file_path="/vsis3/" + filename,
        dep=None,
        date=None,
        n_bands=3,
    )
    bands_indices = [0, 1, 2]
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(np.transpose(si.array, (1, 2, 0))[:, :, bands_indices])

    plt.xticks([])
    plt.yticks([])

    ax.imshow(raster_image, cmap="jet", alpha=0.5)
    plt.show()


# creer un polygon à partir d'une si
def create_square_polygon_from_patch(patch_si):
    xmin, ymin, xmax, ymax = patch_si.bounds

    transformer = Transformer.from_crs("EPSG:4471", "EPSG:4326")

    top_left = (xmin, ymax)
    top_right = (xmax, ymax)
    bottom_right = (xmax, ymin)
    bottom_left = (xmin, ymin)

    square = Polygon([top_left, top_right, bottom_right, bottom_left])

    transformed_corners = [
        (lat, lon)
        for lon, lat in [transformer.transform(*corner) for corner in square.exterior.coords]
    ]

    # Create a new polygon with the transformed coordinates
    square_lat_lon = Polygon(transformed_corners)

    return square_lat_lon

Par la suite : 
- je split l'image en 64 tuiles, je check à chaque tuile si l'ilot intersecte l'image.
Si c'est le cas :
- j'applique la détection sinon je fous des 0 partout en masque

In [ ]:
import torch
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2


# normalized_patch  = preprocess_image(model = model, image = patch)
def predire_mask(patch, model):
    ## preprocess image ne fonctionne pas, je continue à la main-> récupéré sur mlflow
    normalization_mean = [64.16389639532088, 83.11019319058116, 65.75570113248385]
    normalization_std = [27.201775023023046, 29.687673268678918, 31.91543972363831]

    augment_size = 512
    tiles_size = 250

    # Define the transformation pipeline
    transform_list = [
        A.Normalize(
            max_pixel_value=1.0,
            mean=normalization_mean,
            std=normalization_std,
        ),
        ToTensorV2(),
    ]

    # Add resizing transformation if augment size is different from tiles size
    if augment_size != tiles_size:
        transform_list.insert(0, A.Resize(augment_size, augment_size))
        transform = A.Compose(transform_list)

    normalized_si = transform(image=np.transpose(patch.array, [1, 2, 0]))["image"].unsqueeze(dim=0)

    prediction = torch.tensor(model.predict(normalized_si.numpy()))
    mask = produce_mask(prediction, model, patch.array.shape[-2:])
    response = {"mask": mask.tolist()}

    si_mask = SatelliteImage(
        array=np.expand_dims(np.array(response["mask"])[0][1], axis=0),
        crs=patch.crs,
        bounds=patch.bounds,
        transform=patch.transform,
        dep=patch.dep,
        date=patch.date,
    )

    return si_mask

In [ ]:
def calculer_masque_pour_filename_dun_ilot(filename, model, polygon_ilot):
    si = SatelliteImage.from_raster(
        file_path="/vsis3/" + filename,
        dep=None,
        date=None,
        n_bands=3,
    )

    liste_si = si.split(250)

    liste_intersect = [
        create_square_polygon_from_patch(patch).intersects(polygon_ilot) for patch in liste_si
    ]

    liste_si_mask = []
    for i, patch in enumerate(tqdm(liste_si)):
        if liste_intersect[i]:
            si_mask = predire_mask(patch, model)

        else:
            si_mask = SatelliteImage(
                array=np.zeros((1, 250, 250)),
                crs=patch.crs,
                bounds=patch.bounds,
                transform=patch.transform,
                dep=patch.dep,
                date=patch.date,
            )

        liste_si_mask.append(si_mask)

    print("construction mosaique de masque")
    masque_2000_2000 = make_mosaic(liste_si_mask, [0])  # c'est une si

    print("calcul du masque final")
    raster_ilot = construire_raster_ilot(polygon_ilot, filename)

    masque_model_ilot = raster_ilot * masque_2000_2000.array[0]

    si_mask = SatelliteImage(
        array=np.expand_dims(masque_model_ilot, axis=0),
        crs=si.crs,
        bounds=si.bounds,
        transform=si.transform,
        dep=si.dep,
        date=si.date,
    )

    return {"si": si, "si_mask": si_mask}

In [ ]:
def produire_lsi_avec_prev_a_partir_de_code_ilot(depcom, num_ilot, model):
    polygon_ilot = ilots_mayotte.loc[
        (ilots_mayotte.code_ilot == num_ilot) & (ilots_mayotte.depcom == depcom)
    ].geometry.iloc[0]

    filenames_ilot = intersection_ilot_filename.loc[
        (intersection_ilot_filename["code_ilot"] == num_ilot)
        & (intersection_ilot_filename["depcom"] == depcom)
    ].filename

    res_2000_2000 = [
        calculer_masque_pour_filename_dun_ilot(filename, model, polygon_ilot)
        for filename in filenames_ilot
    ]

    si_finale = make_mosaic([res["si"] for res in res_2000_2000], [0, 1, 2])
    masque_final = make_mosaic([res["si_mask"] for res in res_2000_2000], [0]).array[0]

    si_finale.array = si_finale.array.astype(int)
    lsi = SegmentationLabeledSatelliteImage(satellite_image=si_finale, label=masque_final)

    return lsi


# ensuite je prends les masques et j'en fait une mosaic les images pareil et je fais une labelled satellite image

##OK 
Maintenant je me sers de make_mosaic sur la liste des si_mask pour refusionner puis recrée une labeled



In [ ]:
from shapely.geometry import Polygon


def transformer_masque_lsi_en_gepolygon(lsi):
    array = lsi.label
    left, __, __, top = lsi.satellite_image.bounds

    array = array.astype(np.uint16)

    with rasterio.Env():
        with rasterio.open(
            "temp.tif",
            "w+",
            driver="GTiff",
            height=array.shape[0],
            width=array.shape[1],
            count=1,
            dtype=array.dtype,
            nodata=0,
            transform=rasterio.transform.from_origin(left, top, 0.5, 0.5),
        ) as dst:
            dst.write(array, 1)

            results = (
                {"properties": {"raster_val": v}, "geometry": s}
                for i, (s, v) in enumerate(shapes(array, mask=None, transform=dst.transform))
                if v == 1  # Conserver uniquement les amas de 1
            )

    gdf = gpd.GeoDataFrame.from_features(list(results))

    gdf.crs = lsi.satellite_image.crs
    # gdf.to_parquet(f"results/polygons/{filename}_polygons-geo.parquet")
    # gdf = gpd.read_parquet(f"results/polygons/{filename}_polygons-geo.parquet")
    gdf_gps = gdf.to_crs(4326)

    return gdf_gps


def representer_interactivement_pref_ilot(lsi, polygon_ilot):
    gdf_gps = transformer_masque_lsi_en_gepolygon(lsi)

    x, y = polygon_ilot.exterior.coords.xy

    coords = [(yi, xi) for xi, yi in zip(x, y)]
    # Create a map centered around the average location of the polygon
    avg_lon = sum(x) / len(x)
    avg_lat = sum(y) / len(y)
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=15)

    folium.TileLayer(
        tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
        attr="Esri",
        name="Esri Satellite",
        overlay=False,
        control=True,
    ).add_to(m)

    # Convert the GeoDataFrame to GeoJSON (this step may be optional if you can pass the GeoDataFrame directly)
    geojson_data = gdf_gps.to_json()

    # Add the GeoJSON to the Folium map
    folium.GeoJson(
        geojson_data,
        style_function=lambda feature: {
            "fillColor": "#ffff00",
            "color": "yellow",
            "weight": 4,
            #'dashArray': '5, 5'
        },
    ).add_to(m)

    # Add a polygon overlay to the map
    folium.Polygon(locations=coords, color="blue", fill=True, fill_color="cyan").add_to(m)

    return m

In [ ]:
from app.utils import get_model, produce_mask

model = get_model("test", "15")

In [ ]:
# depcom = "97611"
# num_ilot = "0341"

# depcom = "97608"
# num_ilot = "0245"

depcom = "97603"
num_ilot = "0601"
depcom = "97611"
num_ilot = "0242"


lsi = produire_lsi_avec_prev_a_partir_de_code_ilot(depcom, num_ilot, model)
polygon_ilot = ilots_mayotte.loc[
    (ilots_mayotte.code_ilot == num_ilot) & (ilots_mayotte.depcom == depcom)
].geometry.iloc[0]

m = representer_interactivement_pref_ilot(lsi, polygon_ilot)
m.save(depcom + num_ilot + "exemple.html")
m

In [ ]:
## TODO :

##Plus qu'à boucler sur tous les ilots.
# faire des fonctions de représentations ilot, prev contour etc..